## Кривоногов Н.В., PyTorch, практическое задание № 9

Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.

Сделайте предсказания на всем df_val. Посчитайте метрику качества.

Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter

import pandas as pd

In [2]:
!pip install transformers

Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

In [3]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2023-07-19 21:03:25--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 74.125.195.102, 74.125.195.138, 74.125.195.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gbv8r4ea4cpm3dm2bq94mtu7f1feg1id/1689800550000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=c8037afe-e2fa-4c46-80ce-d73b5a1b26d0 [following]
--2023-07-19 21:03:29--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gbv8r4ea4cpm3dm2bq94mtu7f1feg1id/1689800550000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=c8037afe-e2fa-4c46-80ce-d73b5a1b26d0
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 74.125.142.132, 2607:

In [4]:
!unzip data.zip

Archive:  data.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace val.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [5]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [6]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [7]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

### В качестве модели была выбрана s-nlp/russian_toxicity_classifier:

https://huggingface.co/s-nlp/russian_toxicity_classifier

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification

class TwitterDataset(torch.utils.data.Dataset):

    def __init__(self, txts, labels, max_length=10):
        self._labels = labels

        self.tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=max_length,
                                     truncation=True, return_tensors="pt")
                      for text in txts]

    def __len__(self):
        return len(self._txts)

    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [9]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

device

'cuda'

In [11]:
model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier').to(device)

model.eval()
total_acc_val = 0.0
for val_input, val_label in valid_loader:
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze(1).to(device)

    output = model(input_id, mask).logits
    # print(output)

    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc

print(
    f'Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

Val Accuracy:  0.486


На данном этапе метрика равна 48,6%.

In [12]:
#model = XLMClassifier().to(device)
criterion = nn.CrossEntropyLoss()

# optimizer = Adam(model.parameters(), lr=0.001)  # полное обучение
optimizer = Adam(model.classifier.parameters(), lr=0.001)  # неполное обучение

In [13]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.classifier.parameters()]))

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [14]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask).logits
        #output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()

    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask).logits

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()

        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 2836/2836 [04:52<00:00,  9.71it/s]


Epochs: 1 | Train Loss:  0.011         | Train Accuracy:  0.581         | Val Loss:  0.010         | Val Accuracy:  0.610


100%|██████████| 2836/2836 [04:54<00:00,  9.63it/s]


Epochs: 2 | Train Loss:  0.010         | Train Accuracy:  0.594         | Val Loss:  0.010         | Val Accuracy:  0.602


После дообучения модели метрика равна 61%. Рост на 12,4%.